# TinyBERT Keyboard Suggestion Model Training

This notebook trains a keyboard suggestion model using **TinyBERT** with multi-task learning.

**Features:**
1. Word Completion: "Hel" → ["Hello", "Help", "Helping"]
2. Next-Word Prediction: "How are" → ["you", "they", "we"]
3. Typo Correction: "Thers" → ["There", "Theirs", "Therapy"]
4. Gibberish Detection: Heuristic (no ML)

**Model Specifications:**
- Base: TinyBERT (4 layers, 312 hidden, 4 heads, ~14M params)
- Target Size: <5MB (after INT8 quantization)
- Latency: <50ms on mobile
- RAM Usage: <30MB runtime
- Deployment: iOS (CoreML) + Android (TFLite)

**Training Time:** 2-4 hours on Colab GPU (T4)

**Data Sources (Google Drive):**
- `single_word_freq.csv` - Word frequencies for completion
- `keyboard_training_data.txt` - Custom corpus for next-word
- `misspelled.csv` - Typo correction pairs

---

**Instructions:**
1. Runtime → Change runtime type → GPU (T4)
2. Run all cells
3. Model will be saved to Google Drive
4. Download for mobile deployment

## 1. Environment Setup

In [ ]:
# Check if running in Colab
import os

IN_COLAB = 'COLAB_GPU' in os.environ or 'COLAB_TPU_ADDR' in os.environ

if IN_COLAB:
    print("✓ Running in Google Colab")
    
    # Mount Google Drive
    from google.colab import drive
    drive.mount('/content/drive')
    
    # Define Drive directory
    DRIVE_DIR = '/content/drive/MyDrive/Keyboard-Suggestions-ML-Colab'
    
    # Create directories
    os.makedirs(DRIVE_DIR, exist_ok=True)
    os.makedirs(f"{DRIVE_DIR}/datasets", exist_ok=True)
    os.makedirs(f"{DRIVE_DIR}/datasets/processed", exist_ok=True)
    os.makedirs(f"{DRIVE_DIR}/models", exist_ok=True)
    
    print(f"✓ Google Drive mounted")
    print(f"✓ Project directory: {DRIVE_DIR}")
else:
    print("✓ Running locally")
    DRIVE_DIR = './data'  # Local fallback
    os.makedirs(DRIVE_DIR, exist_ok=True)
    os.makedirs(f"{DRIVE_DIR}/datasets", exist_ok=True)
    os.makedirs(f"{DRIVE_DIR}/datasets/processed", exist_ok=True)
    os.makedirs(f"{DRIVE_DIR}/models", exist_ok=True)

In [ ]:
# Install dependencies
!pip install -q transformers torch datasets accelerate
!pip install -q scikit-learn tqdm
print("✓ Dependencies installed")

## 2. Verify Datasets in Google Drive

**Expected datasets in Google Drive:**
- `{DRIVE_DIR}/datasets/single_word_freq.csv`
- `{DRIVE_DIR}/datasets/keyboard_training_data.txt`
- `{DRIVE_DIR}/datasets/misspelled.csv`

In [ ]:
import os

print("Checking datasets in Google Drive...")
print("="*60)

# Define dataset paths
WORD_FREQ_PATH = f"{DRIVE_DIR}/datasets/single_word_freq.csv"
CORPUS_PATH = f"{DRIVE_DIR}/datasets/keyboard_training_data.txt"
TYPO_PATH = f"{DRIVE_DIR}/datasets/misspelled.csv"

# Check each dataset
datasets_ok = True

if os.path.exists(WORD_FREQ_PATH):
    with open(WORD_FREQ_PATH, 'r', encoding='utf-8') as f:
        word_count = sum(1 for _ in f) - 1  # Subtract header
    print(f"✓ single_word_freq.csv: {word_count:,} words")
else:
    print(f"✗ Missing: {WORD_FREQ_PATH}")
    datasets_ok = False

if os.path.exists(CORPUS_PATH):
    with open(CORPUS_PATH, 'r', encoding='utf-8') as f:
        line_count = sum(1 for _ in f)
    print(f"✓ keyboard_training_data.txt: {line_count:,} lines")
else:
    print(f"✗ Missing: {CORPUS_PATH}")
    datasets_ok = False

if os.path.exists(TYPO_PATH):
    with open(TYPO_PATH, 'r', encoding='utf-8') as f:
        typo_count = sum(1 for _ in f) - 1  # Subtract header
    print(f"✓ misspelled.csv: {typo_count:,} entries")
else:
    print(f"✗ Missing: {TYPO_PATH}")
    datasets_ok = False

print("="*60)
if datasets_ok:
    print("✅ All datasets found!")
else:
    print("⚠️  Some datasets are missing. Please upload them to Google Drive.")
    print("\nExpected location: {DRIVE_DIR}/datasets/")
    print("Required files:")
    print("  - single_word_freq.csv (format: word,count_frequency)")
    print("  - keyboard_training_data.txt (plain text sentences)")
    print("  - misspelled.csv (format: number,correct_word,misspelled_words)")

## 3. Generate Training Data

Generate training pairs for all 3 tasks from your existing datasets

In [ ]:
import json
import random
import csv
from typing import List, Tuple

random.seed(42)

def prepare_word_completion_data(word_freq_path: str, max_samples: int = 50000) -> List[dict]:
    """Generate word completion training pairs from single_word_freq.csv"""
    print("\nGenerating word completion data...")
    
    samples = []
    words_with_freq = []
    
    # Read words with frequencies
    with open(word_freq_path, 'r', encoding='utf-8') as f:
        reader = csv.DictReader(f)
        for row in reader:
            word = row['word'].strip().lower()
            freq = int(row.get('count_frequency', 1))
            if len(word) >= 3:  # Only words with 3+ chars
                words_with_freq.append((word, freq))
    
    # Sort by frequency (higher first)
    words_with_freq.sort(key=lambda x: x[1], reverse=True)
    
    # Generate samples (weighted by frequency)
    for word, freq in words_with_freq:
        if len(samples) >= max_samples:
            break
        
        # Generate multiple prefixes for common words
        num_samples = min(3, max(1, freq // 1000))  # More samples for frequent words
        
        for _ in range(num_samples):
            if len(samples) >= max_samples:
                break
            
            # Random prefix length (50-80% of word)
            prefix_len = random.randint(max(1, len(word) // 2), max(2, int(len(word) * 0.8)))
            prefix = word[:prefix_len]
            
            samples.append({
                'input': prefix,
                'target': word,
                'task': 'completion'
            })
    
    print(f"  Generated {len(samples):,} completion pairs")
    return samples

def prepare_nextword_data(corpus_path: str, max_samples: int = 100000, context_length: int = 3) -> List[dict]:
    """Generate next-word prediction pairs from keyboard_training_data.txt"""
    print("\nGenerating next-word prediction data...")
    
    samples = []
    
    with open(corpus_path, 'r', encoding='utf-8') as f:
        for line in f:
            if len(samples) >= max_samples:
                break
            
            line = line.strip().lower()
            words = line.split()
            
            # Skip short sentences
            if len(words) < context_length + 1:
                continue
            
            # Generate multiple samples from each sentence
            for i in range(len(words) - context_length):
                context = ' '.join(words[i:i+context_length])
                target = words[i+context_length]
                
                # Filter out punctuation-only targets
                if target.isalpha() and len(target) > 1:
                    samples.append({
                        'input': context,
                        'target': target,
                        'task': 'nextword'
                    })
                
                if len(samples) >= max_samples:
                    break
    
    print(f"  Generated {len(samples):,} next-word pairs")
    return samples

def prepare_typo_data(typo_path: str, max_samples: int = 20000) -> List[dict]:
    """Generate typo correction pairs from misspelled.csv"""
    print("\nGenerating typo correction data...")
    
    samples = []
    
    with open(typo_path, 'r', encoding='utf-8') as f:
        reader = csv.DictReader(f)
        for row in reader:
            if len(samples) >= max_samples:
                break
            
            correct = row['correct_word'].strip().lower()
            misspelled_list = row['misspelled_words'].strip().lower()
            
            # Split multiple misspellings (comma or space separated)
            typos = [t.strip() for t in misspelled_list.replace(',', ' ').split() if t.strip()]
            
            for typo in typos:
                if len(samples) >= max_samples:
                    break
                
                if typo and typo != correct:
                    samples.append({
                        'input': typo,
                        'target': correct,
                        'task': 'typo'
                    })
    
    print(f"  Generated {len(samples):,} typo pairs")
    return samples

# Generate all datasets
print("Preparing training datasets...")
print("="*60)

output_dir = f"{DRIVE_DIR}/datasets/processed"
os.makedirs(output_dir, exist_ok=True)

train_path = f"{output_dir}/train.jsonl"
val_path = f"{output_dir}/val.jsonl"

# Check if processed datasets already exist
if os.path.exists(train_path) and os.path.exists(val_path):
    print("✓ Processed datasets found in Drive!")
    print(f"  Train: {train_path}")
    print(f"  Val: {val_path}")
    
    # Count samples
    with open(train_path, 'r') as f:
        train_count = sum(1 for _ in f)
    with open(val_path, 'r') as f:
        val_count = sum(1 for _ in f)
    print(f"  Train samples: {train_count:,}")
    print(f"  Val samples: {val_count:,}")
else:
    print("Generating training datasets from scratch...")
    
    # Generate each task
    completion_samples = prepare_word_completion_data(WORD_FREQ_PATH, max_samples=50000)
    nextword_samples = prepare_nextword_data(CORPUS_PATH, max_samples=100000, context_length=3)
    typo_samples = prepare_typo_data(TYPO_PATH, max_samples=20000)
    
    # Combine all samples
    all_samples = completion_samples + nextword_samples + typo_samples
    random.shuffle(all_samples)
    
    # Split train/val (90/10)
    split_idx = int(len(all_samples) * 0.9)
    train_samples = all_samples[:split_idx]
    val_samples = all_samples[split_idx:]
    
    # Save to JSONL
    with open(train_path, 'w', encoding='utf-8') as f:
        for sample in train_samples:
            f.write(json.dumps(sample) + '\n')
    
    with open(val_path, 'w', encoding='utf-8') as f:
        for sample in val_samples:
            f.write(json.dumps(sample) + '\n')
    
    print("\n" + "="*60)
    print("✓ Dataset generation complete!")
    print(f"  Total samples: {len(all_samples):,}")
    print(f"  Train: {len(train_samples):,} ({train_path})")
    print(f"  Val: {len(val_samples):,} ({val_path})")
    print(f"\n  Task distribution:")
    print(f"    Completion: {len(completion_samples):,}")
    print(f"    Next-word: {len(nextword_samples):,}")
    print(f"    Typo: {len(typo_samples):,}")

print("\n" + "="*60)
print("✓ Datasets ready for training!")

## 4. Load TinyBERT Model and Tokenizer

In [ ]:
from transformers import AutoTokenizer, AutoModelForMaskedLM
import torch

print("Loading TinyBERT for Masked Language Modeling...")
print("="*60)

# Load TinyBERT (4 layers, 312 hidden, 4 heads, ~14M params)
MODEL_NAME = "google/bert_uncased_L-4_H-312_A-12"

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForMaskedLM.from_pretrained(MODEL_NAME)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

print(f"✓ Model loaded on {device}")
print(f"  Model: {MODEL_NAME}")
print(f"  Parameters: {sum(p.numel() for p in model.parameters()):,}")
print(f"  Layers: 4, Hidden: 312, Heads: 12")
print(f"  Model size: ~55MB (FP32) → ~5MB (INT8 quantized)")
print(f"  Target latency: <50ms on mobile")
print(f"  Target RAM: <30MB runtime")

## 5. Prepare Training Data

In [ ]:
from torch.utils.data import Dataset, DataLoader
import json
import torch

class KeyboardDataset(Dataset):
    """BERT MLM dataset for keyboard suggestions."""
    
    def __init__(self, data_path, tokenizer, max_length=16):
        self.data = []
        with open(data_path, 'r', encoding='utf-8') as f:
            for line in f:
                self.data.append(json.loads(line))
        self.tokenizer = tokenizer
        self.max_length = max_length
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        item = self.data[idx]
        text_input = item['input']
        target_word = item['target']
        task = item.get('task', 'completion')
        
        # Add [MASK] for prediction
        # - Completion: "hel [MASK]" → predict "lo" (hello)
        # - Next-word: "how are [MASK]" → predict "you"
        # - Typo: "thers [MASK]" → predict "there"
        text_input = f"{text_input} {self.tokenizer.mask_token}"
        
        # Tokenize
        inputs = self.tokenizer(
            text_input,
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )
        
        # Labels: -100 everywhere except [MASK]
        labels = torch.full(inputs['input_ids'].shape, -100, dtype=torch.long)
        
        # Get target token ID (first token of target word)
        target_tokens = self.tokenizer.tokenize(target_word)
        target_id = self.tokenizer.convert_tokens_to_ids(target_tokens[0]) if target_tokens else self.tokenizer.unk_token_id
        
        # Set label at [MASK] position
        mask_positions = (inputs['input_ids'] == self.tokenizer.mask_token_id).nonzero(as_tuple=True)
        if len(mask_positions[1]) > 0:
            labels[0, mask_positions[1][0]] = target_id
        
        return {
            'input_ids': inputs['input_ids'].squeeze(),
            'attention_mask': inputs['attention_mask'].squeeze(),
            'labels': labels.squeeze(),
            'task': task
        }

print("Loading training data...")
train_dataset = KeyboardDataset(train_path, tokenizer, max_length=16)
val_dataset = KeyboardDataset(val_path, tokenizer, max_length=16)

print(f"✓ Train samples: {len(train_dataset):,}")
print(f"✓ Val samples: {len(val_dataset):,}")

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)

print(f"✓ Batch size: 32 (train), 64 (val)")
print(f"✓ Max sequence length: 16 tokens")

## 6. Training

In [ ]:
from torch.optim import AdamW
from tqdm.auto import tqdm

NUM_EPOCHS = 3
LEARNING_RATE = 3e-5  # Lower LR for fine-tuning
SAVE_STEPS = 1000

optimizer = AdamW(model.parameters(), lr=LEARNING_RATE)

print("Starting training...")
print("="*60)
print(f"Epochs: {NUM_EPOCHS}")
print(f"Learning rate: {LEARNING_RATE}")
print(f"Optimizer: AdamW")
print(f"Save checkpoints every: {SAVE_STEPS} steps")
print("="*60)

global_step = 0
best_val_loss = float('inf')

for epoch in range(NUM_EPOCHS):
    print(f"\nEpoch {epoch + 1}/{NUM_EPOCHS}")
    
    model.train()
    train_loss = 0
    
    progress_bar = tqdm(train_loader, desc="Training")
    for batch in progress_bar:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item()
        global_step += 1
        progress_bar.set_postfix({'loss': loss.item()})
        
        if global_step % SAVE_STEPS == 0:
            checkpoint_dir = f"{DRIVE_DIR}/models/checkpoint-{global_step}"
            model.save_pretrained(checkpoint_dir)
            tokenizer.save_pretrained(checkpoint_dir)
            print(f"\n✓ Checkpoint saved: {checkpoint_dir}")
    
    avg_train_loss = train_loss / len(train_loader)
    print(f"  Train loss: {avg_train_loss:.4f}")
    
    # Validation
    model.eval()
    val_loss = 0
    
    with torch.no_grad():
        for batch in tqdm(val_loader, desc="Validation"):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)
            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            val_loss += outputs.loss.item()
    
    avg_val_loss = val_loss / len(val_loader)
    print(f"  Val loss: {avg_val_loss:.4f}")
    
    # Save best model
    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        best_model_dir = f"{DRIVE_DIR}/models/best_model"
        model.save_pretrained(best_model_dir)
        tokenizer.save_pretrained(best_model_dir)
        print(f"  ✓ Best model saved: {best_model_dir}")

print("\n" + "="*60)
print("✓ Training complete!")
print(f"  Best val loss: {best_val_loss:.4f}")
print(f"  Total steps: {global_step}")

## 7. Export and Save Model

In [ ]:
# Export and save the BEST model (for iOS/Android deployment)
import shutil
from google.colab import files

print("Exporting BEST model for deployment...")
print("="*60)

# Load the best model (saved during training)
best_model_path = f"{DRIVE_DIR}/models/best_model"
print(f"\nLoading best model from: {best_model_path}")

# Load best model
best_model = AutoModelForMaskedLM.from_pretrained(best_model_path)
best_tokenizer = AutoTokenizer.from_pretrained(best_model_path)
print("✓ Best model loaded")

# 1. Save to Google Drive as 'final' (for deployment)
drive_model_dir = f"{DRIVE_DIR}/models/tinybert_keyboard_final"
best_model.save_pretrained(drive_model_dir)
best_tokenizer.save_pretrained(drive_model_dir)
print(f"\n✅ Saved to Google Drive: {drive_model_dir}")

# 2. Create downloadable zip
local_model_dir = "/content/tinybert_keyboard_model"
best_model.save_pretrained(local_model_dir)
best_tokenizer.save_pretrained(local_model_dir)

zip_path = "/content/tinybert_keyboard_model.zip"
shutil.make_archive("/content/tinybert_keyboard_model", 'zip', local_model_dir)
print(f"\n✅ Created zip: {zip_path}")

# 3. Download to local device
if IN_COLAB:
    print("\n📥 Downloading BEST model to your computer...")
    files.download(zip_path)
    print("✅ Download started! Check your Downloads folder.")

print("\n" + "="*60)
print("✅ Export complete!")
print("\n⚠️  IMPORTANT: This is the BEST model (lowest validation loss)")
print("   Use this for iOS/Android deployment!")
print("\nModel saved to:")
print(f"  1. Google Drive: {drive_model_dir}")
print(f"  2. Local download: tinybert_keyboard_model.zip")
print("\nNext steps:")
print("  1. Run Section 8 (Export to CoreML for iOS)")
print("  2. Run Section 9 (Export to TFLite for Android)")
print("\n💡 Both iOS and Android exports will use this BEST model!")

## 8. Export to CoreML (iOS)

In [ ]:
# Export to CoreML for iOS with INT8 quantization
!pip install -q coremltools

import coremltools as ct
from coremltools.models.neural_network import quantization_utils
import torch

print("Exporting to CoreML for iOS...")
print("="*60)

# 1. Prepare model for tracing
print("\n1. Preparing model for conversion...")
best_model.cpu().eval()

# Create dummy inputs (batch_size=1, seq_length=16)
dummy_input_ids = torch.zeros(1, 16, dtype=torch.long)
dummy_attention_mask = torch.ones(1, 16, dtype=torch.long)

# 2. Trace the model
print("2. Tracing model...")
traced_model = torch.jit.trace(best_model, (dummy_input_ids, dummy_attention_mask))
print("   ✓ Model traced successfully")

# 3. Convert to CoreML (FP32)
print("\n3. Converting to CoreML (FP32)...")
mlmodel = ct.convert(
    traced_model,
    inputs=[
        ct.TensorType(name="input_ids", shape=(1, 16), dtype=int),
        ct.TensorType(name="attention_mask", shape=(1, 16), dtype=int)
    ],
    outputs=[ct.TensorType(name="logits")]
)
print("   ✓ Conversion successful")

# 4. Quantize to INT8
print("\n4. Quantizing to INT8...")
mlmodel_int8 = quantization_utils.quantize_weights(mlmodel, nbits=8)
print("   ✓ Quantization complete")

# 5. Save to Drive
coreml_path = f"{DRIVE_DIR}/models/TinyBERT_Keyboard_iOS.mlpackage"
mlmodel_int8.save(coreml_path)

print("\n" + "="*60)
print("✅ iOS CoreML export complete!")
print(f"\nSaved to: {coreml_path}")
print(f"Model size: ~5MB (INT8 quantized)")
print(f"Expected latency: <50ms on iPhone (with Neural Engine)")
print(f"Expected RAM: <30MB runtime")
print("\nNext steps:")
print("  1. Download from Google Drive")
print("  2. Add to Xcode project")
print("  3. Use with Core ML framework")
print("\n💡 Tip: Enable Neural Engine for best performance!")

## 9. Export to TFLite (Android)

In [ ]:
# Export to TFLite for Android with INT8 quantization
!pip install -q tensorflow onnx onnx-tf

import tensorflow as tf
from pathlib import Path
import onnx
from onnx_tf.backend import prepare

print("Exporting to TFLite for Android...")
print("="*60)

# 1. Convert PyTorch to ONNX first
print("\n1. Converting PyTorch → ONNX...")
best_model.cpu().eval()

onnx_path = "/content/model.onnx"
dummy_input_ids = torch.zeros(1, 16, dtype=torch.long)
dummy_attention_mask = torch.ones(1, 16, dtype=torch.long)

torch.onnx.export(
    best_model,
    (dummy_input_ids, dummy_attention_mask),
    onnx_path,
    input_names=['input_ids', 'attention_mask'],
    output_names=['logits'],
    dynamic_axes={
        'input_ids': {0: 'batch_size'},
        'attention_mask': {0: 'batch_size'},
        'logits': {0: 'batch_size'}
    },
    opset_version=18
)
print("   ✓ ONNX export successful")

# 2. Convert ONNX to TensorFlow SavedModel
print("\n2. Converting ONNX → TensorFlow...")
onnx_model = onnx.load(onnx_path)
tf_rep = prepare(onnx_model)
tf_model_path = "/content/tf_model"
tf_rep.export_graph(tf_model_path)
print("   ✓ TensorFlow conversion successful")

# 3. Convert to TFLite with INT8 quantization
print("\n3. Converting to TFLite with INT8 quantization...")
converter = tf.lite.TFLiteConverter.from_saved_model(tf_model_path)
converter.optimizations = [tf.lite.Optimize.DEFAULT]  # INT8 quantization
tflite_model = converter.convert()
print("   ✓ TFLite conversion successful")

# 4. Save to Drive
tflite_dir = Path(f"{DRIVE_DIR}/models/tflite")
tflite_dir.mkdir(parents=True, exist_ok=True)

tflite_path = tflite_dir / "keyboard_model_quantized.tflite"
with open(tflite_path, "wb") as f:
    f.write(tflite_model)

# 5. Save vocabulary
best_tokenizer.save_vocabulary(str(tflite_dir))

import os
model_size_mb = os.path.getsize(tflite_path) / 1024 / 1024

print("\n" + "="*60)
print("✅ Android TFLite export complete!")
print(f"\nSaved to: {tflite_path}")
print(f"Model size: {model_size_mb:.2f}MB (INT8 quantized)")
print(f"Expected latency: <50ms on Android (with NPU delegate)")
print(f"Expected RAM: <30MB runtime")
print("\nFiles created:")
print(f"  • keyboard_model_quantized.tflite")
print(f"  • vocab.txt")
print("\nNext steps:")
print("  1. Download from Google Drive")
print("  2. Add to Android project (assets/)")
print("  3. Use with TensorFlow Lite Interpreter")
print("\n💡 Tip: Enable GPU/NPU delegate for best performance!")